In [1]:
import sys
sys.path.append("D:/File/大/大三下/poke")
# 导入 Flask 框架、request 对象和 secure_filename 方法
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import os
from Processing import Processing
from Predict import predict
from flask_cors import *
import math


In [2]:
# 允许上传的文件类型，这里限制为 csv、xlsx 和 xls 格式的文件
ALLOWED_EXTENSIONS = set(['csv', 'xlsx', 'xls'])

# 检查上传的文件类型是否在 ALLOWED_EXTENSIONS 中
def isAllowedFile(filename):
    return '.'in filename and filename.split('.')[-1] in ALLOWED_EXTENSIONS

In [3]:
app=Flask(__name__)
# 设置文件上传保存路径，这里设置为当前文件夹下的 upload 文件夹
app.config['UPLOAD_FOLDER'] = 'upload/'
CORS(app, supports_credentials=True)

In [4]:
@app.route('/load_network',methods=['GET'])
def load_network():
    with open("data_network.json",'r', encoding='UTF-8') as f:
        load_dict = f.read()
    return load_dict

In [5]:
@app.route('/load_wordcloud',methods=['GET'])
def load_wordcloud():
    with open("data_wordcloud.json",'r', encoding='UTF-8') as f:
        load_dict = f.read()
    return load_dict

In [6]:
@app.route('/load_flow',methods=['GET'])
def load_flow():
    with open("data_flow.json",'r', encoding='UTF-8') as f:
        load_dict = f.read()
    return load_dict

In [7]:
@app.route('/load_line',methods=['POST'])
def load_line():
    lst=request.get_json()

    import json
    with open('line_data.json','r', encoding='UTF-8')as f:
        data=json.load(f)
    ans=[]
    for hosptial in lst: 
        ans.append(data[hosptial])
    return json.dumps({k: data[k] for k in lst}, ensure_ascii=False)

In [8]:
@app.route('/load_hospital',methods=['GET'])
def load_hospital():
    import json
    with open('line_hospital.json','r', encoding='UTF-8')as f:
        data=json.load(f)
    data = list(set(data))
    data = [d for d in data if type(d) is str]
    # data.remove(float('nan'))
    return json.dumps(data, ensure_ascii=False)

In [9]:
@app.route('/upload', methods=['POST'])
def upload_file():
    # 处理文件上传请求
    # if request.method == 'POST':
        # 判断请求中是否包含文件
        if 'file' not in request.files:
            return {'message': 'No file part'}
        file = request.files['file']

        # 如果用户没有选择文件，则返回错误信息
        if file.filename == '':
            return {'message': 'No selected file'}

        # 如果用户上传的文件符合要求，则保存文件并返回成功信息
        if file and isAllowedFile(file.filename):
            # 使用 secure_filename 方法生成安全文件名
            filename = secure_filename(file.filename)
            # 将文件保存到指定目录
            originPath=os.path.join(app.config['UPLOAD_FOLDER'], filename)
            processedPath=os.path.join(app.config['UPLOAD_FOLDER'], "processed"+filename.split('.')[0]+".csv")
            file.save(originPath)
            # 数据预处理并预测
            try:
                print("start process data...")
                prcs=Processing()
                prcs.processing(originPath,processedPath)
                print("process data overed")
            except: 
                print("数据格式不匹配")
                return {'error':"数据格式不匹配"}
            ans=predict(processedPath)
            print(ans)
            return {'message': ans}
            
            return jsonify({'message': 'File uploaded successfully'})

In [10]:
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
[2023-06-10 15:26:49,026] ERROR in app: Exception on /load_hospital [GET]
Traceback (most recent call last):
  File "/opt/anaconda/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/opt/anaconda/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/meo/.local/lib/python3.10/site-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/opt/anaconda/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/opt/anaconda/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/tmp/ipykernel_54797/4099681.py", line 7, in load_hospital
    d